In [148]:
import numpy as np
import pandas as pd
import pickle

import geopandas as gpd
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from os import path
from shapely.geometry import Point
from shapely.ops import unary_union
from shapely import wkt
from tqdm import tqdm

In [149]:
geolocator = Nominatim(user_agent='housing-qc')

In [150]:
bounding_territories_df = pd.read_csv('../data/references/handmade/bounding-territories.csv')
bounding_territories_df.sample(5)

,Bounding Territory,Display Name,Bounding Type,Bounding Population,GeoPy Index
34,Brome-Missisquoi,Brome-Missisquoi,Regional County Municipality (RCM),60000,0
84,"[La Tuque,Mékinac,Les Chenaux]","La Tuque, Mékinac & Les Chenaux",Regional County Municipality (RCM),40000,0
43,"[Les Jardins-de-Napierville,Le Haut-Saint-Laur...",Les Jardins-de-Napierville & Le Haut-Saint-Lau...,Regional County Municipality (RCM),50000,0
24,Coaticook (MRC),Coaticook,Regional County Municipality (RCM),20000,0
19,Lotbinière,Lotbinière,Regional County Municipality (RCM),30000,1


In [151]:
def get_bounding_polygons(geolocator: Nominatim, bounding: str, geopy_index: int):
    polygons = []
    substracts = []
    for location in bounding.replace("[", "").replace("]", "").split(","):
        if location.startswith("-"):
            geocodes = geolocator.geocode(location[1:] + ', QC', geometry='wkt', exactly_one=False)
            for geocode in geocodes:
                substracts.append(wkt.loads(geocode.raw['geotext']))
        else:
            geocode = geolocator.geocode(location + ', QC', geometry='wkt', exactly_one=False)[geopy_index]
            polygons.append(wkt.loads(geocode.raw['geotext']))

    return polygons, substracts

In [152]:
def get_bounding_polygon(geolocator: Nominatim, bounding: str, geopy_index: int):
    polygons, substracts = get_bounding_polygons(geolocator, bounding, geopy_index)
    return gpd.GeoSeries(unary_union(polygons).difference(unary_union(substracts))).simplify(tolerance=0.001).iloc[0]

In [153]:
def get_polygons(geolocator: Nominatim, bounding_territories_df: pd.DataFrame):
    polygons = []
    for _, location in tqdm(bounding_territories_df.iterrows(), desc="Building GeoSeries", total=bounding_territories_df.shape[0]):
        polygons.append(get_bounding_polygon(geolocator, location["Bounding Territory"], location["GeoPy Index"]))
    return polygons

In [154]:
output = '../data/processed/locations/location-polygons.gpkg'

if path.exists(output):
    unknown_locations = []
    polygons_gdf = gpd.read_file(output)
else:
    polygons = get_polygons(geolocator, bounding_territories_df)
    d = {'location': bounding_territories_df["Display Name"].to_list(), 'geometry': polygons}
    polygons_gdf = gpd.GeoDataFrame(d, crs="EPSG:4326")
    polygons_gdf.to_file(output)

Building GeoSeries: 100%|██████████| 110/110 [01:11<00:00,  1.53it/s]


In [156]:
polygons_gdf.sample(10)

,location,geometry
26,Bécancour & Nicolet,"POLYGON ((-72.21853 46.18586, -72.21888 46.170..."
83,Trois-Rivières,"POLYGON ((-72.77942 46.31866, -72.77905 46.316..."
56,Beaconsfield,"POLYGON ((-73.90118 45.43481, -73.89409 45.409..."
20,Sherbrooke,"POLYGON ((-72.10879 45.30149, -72.04219 45.299..."
48,"Duvernay, Saint-Vincent-de-Paul & Saint-François","POLYGON ((-73.63864 45.62648, -73.63874 45.624..."
54,Ahuntsic-Cartierville,"MULTIPOLYGON (((-73.76113 45.51047, -73.75702 ..."
47,Chomedey,"POLYGON ((-73.81391 45.55635, -73.80087 45.542..."
7,La Haute-Saint-Charles,"POLYGON ((-71.54922 46.85118, -71.53139 46.838..."
102,Thérèse-De Blainville,"POLYGON ((-73.93177 45.63979, -73.88043 45.606..."
78,"Senneville, Baie-D'Urfé & Saint-Anne-de-Bellevue","POLYGON ((-73.90125 45.43513, -73.89409 45.409..."
